In [ ]:
#All imports used throughout testing
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import lightkurve as lk
import astropy.units as u
import tkinter as tk
from tkinter import ttk
from matplotlib.figure import Figure
import matplotlib
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import csv
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.pipeline import Pipeline
import seaborn as sns
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import silhouette_score, silhouette_samples
import warnings
from sklearn.exceptions import ConvergenceWarning
from datetime import datetime

In [ ]:
#This function separates a pandas dataframe into separate smaller ones by variable star type using the 'Type' column from VisieR https://vizier.cds.unistra.fr/viz-bin/VizieR-3
#The types focused on for this project are 'MISC' (miscellaneous), 'CST' (constant), 'RR' (RR Lyrae), 'EC' (contact eclipsing binary system), 'ED' (detached eclipsing binary system), 'DSCT' (delta scuti), 'DCEP' (classical cepheid/delta cephei-type variables), 'GDOR' (gamma doradus)
#Subtypes listed in "VARIABLE STAR TYPE DESIGNATIONS IN VSX" https://www.aavso.org/vsx/index.php?view=about.vartypes are included in these general categories

def septypes(dataset):  #Input a pandas dataframe that includes a 'Type' column from VisieR
    nmisc = []  #Initializing empty lists where the indices where there is not a specific star type will be appended, for masking
    ncst = []
    nrr = []
    ned = []
    nec = []
    ndsct = []
    ndcep = []
    ngdor = []

    for i in range(len(dataset['Type'])):     #Loops through each index in the dataset
        if 'MISC' not in dataset['Type'][i]:   #If a specific star type or any of its subtypes are not found in the 'Type',the index will be appended to the corresponding list
            nmisc.append(i)

        if 'CST' not in dataset['Type'][i]:
            ncst.append(i)

        if 'RR' not in dataset['Type'][i]:
            nrr.append(i)

        if 'EC' not in dataset['Type'][i]:
            if 'EW' not in dataset['Type'][i]:
                nec.append(i)

        if 'ED' not in dataset['Type'][i]:
            if 'EA' not in dataset['Type'][i]:
                if 'ESD' not in dataset['Type'][i]:
                    if 'EB' not in dataset['Type'][i]:
                        ned.append(i)

        if 'DSCT' not in dataset['Type'][i]:
            if 'HADS' not in dataset['Type'][i]:
                ndsct.append(i)

        if 'DCEP' not in dataset['Type'][i]:
            ndcep.append(i)
        
        if 'GDOR' not in dataset['Type'][i]:
            ngdor.append(i)

    MISC = dataset.drop(nmisc)  #Dropping all indices where the 'Type' doesn't match for each category
    CST = dataset.drop(ncst)
    RR = dataset.drop(nrr)
    EC = dataset.drop(nec)
    ED = dataset.drop(ned)
    DSCT = dataset.drop(ndsct)
    DCEP = dataset.drop(ndcep)
    GDOR = dataset.drop(ngdor)

    for val in MISC.Type.unique():  #Ensuring for each category that all the 'Type' names are the same for machine learning purposes
        MISC.replace(to_replace=val, value='MISC', inplace=True)  #This could probably be consolidated into a smaller chunk of code with a bit of time
    MISC = MISC.reset_index()
    MISC = MISC.drop('index',axis = 1)

    for val in CST.Type.unique():
        CST.replace(to_replace=val, value='CST', inplace=True)
    CST = CST.reset_index()
    CST = CST.drop('index',axis = 1)

    for val in RR.Type.unique():
        RR.replace(to_replace=val, value='RR', inplace=True)
    RR = RR.reset_index()
    RR = RR.drop('index',axis = 1)

    for val in EC.Type.unique():
        EC.replace(to_replace=val, value='EC', inplace=True)
    EC = EC.reset_index()
    EC = EC.drop('index',axis = 1)

    for val in ED.Type.unique():
        ED.replace(to_replace=val, value='ED', inplace=True)
    ED = ED.reset_index()
    ED = ED.drop('index',axis = 1)

    for val in DSCT.Type.unique():
        DSCT.replace(to_replace=val, value='DSCT', inplace=True)
    DSCT = DSCT.reset_index()
    DSCT = DSCT.drop('index',axis = 1)

    for val in DCEP.Type.unique():
        DCEP.replace(to_replace=val, value='DCEP', inplace=True)
    DCEP = DCEP.reset_index()
    DCEP = DCEP.drop('index',axis = 1)

    for val in GDOR.Type.unique():
        GDOR.replace(to_replace=val, value='GDOR', inplace=True)
    GDOR = GDOR.reset_index()
    GDOR = GDOR.drop('index',axis = 1)

    return(MISC,CST,RR,EC,ED,DSCT,DCEP,GDOR)  #Returns separate pandas dataframes containing only the specific type of star that correlates with the variable name

In [ ]:
#This function allows you to clean a dataset that includes one "Type" by returning a csv file that includes the word "yes" or "no" for each index in the dataset depending on which button is pressed. When 'yes' or the y key is pressed, the lightcurve of will be saved as a jpeg file
#Pandas "dataset" must have columns 'TIC_ID','Sector','Camera','CCD', and 'Type' ('Type' must all be the same type)
#'newfilename' is the name that you want the CSV to be saved as
#'foldername' is the name of the folder that you want the created folder of lightcurves to go into
#'seccam' is a name for the sector and camera that the dataset is from, example 'sec15cam3'
#max and min are the maximum and minimum period values to have a window pop up, if the period is outside of these values, 'no' will be appended to the csv file
#Right now, only sectors 1,2,3,4,5,15,16, and 17 will work with this function
def clean(dataset,newfilename,foldername,seccam,max,min):  

    cleaned = [] #Initializing an empty list where 'yes' or 'no' will be appended depending on whether or not a light curve was labelled correctly 

    for i in range(len(dataset['TIC_ID'])): #Looping to get a light curve for each index in the dataset
    
        TICID = dataset['TIC_ID'][i]  
        SECTOR = dataset['Sector'][i]
        CAMERA = dataset['Camera'][i]
        CCD = dataset['CCD'][i]
        
        #A different url is used to download the lightcurve data for sectors 1-5 vs sectors 15-17
        if SECTOR in ['sector01','sector02','sector03','sector04','sector05']:  
            url = f"http://astro.phy.vanderbilt.edu/~oelkerrj/tess_ffi/{SECTOR}/clean/{TICID}_{SECTOR}_{CAMERA}_{CCD}.lc"
        if SECTOR in ['sector15','sector16','sector17']:
            url = f"http://astro.phy.vanderbilt.edu/~oelkerrj/tess_ffi/{SECTOR}/detrend/{TICID}_{SECTOR}_{CAMERA}_{CCD}.lc"

        os.system(f"wget {url} -O {TICID}_{SECTOR}_{CAMERA}_{CCD}.lc > /dev/null 2>&1") #Collecting light curve data for the index
        data = pd.read_csv(f"{TICID}_{SECTOR}_{CAMERA}_{CCD}.lc",delimiter=' ',header=None,names=['time','magnitude','error'])
        data = data.to_numpy(dtype=float)
    
        if np.any(np.isnan(data)) or np.any(np.isinf(data)):  #Checking if there are any nan or inf in the light curve data and skipping the index if there are
            cleaned.append('no')
            os.remove(f"{TICID}_{SECTOR}_{CAMERA}_{CCD}.lc")
            continue
            
        time = data[:,0]
        magnitude = data[:,1]
        
        lc = lk.LightCurve(time,magnitude).remove_outliers(sigma=5.0,return_mask = False) #
        pg = lc.to_periodogram(minimum_period=0.025*u.day, maximum_period=20*u.day,oversample_factor=10) #Using lightkurve to find the period with max power using a lomb-scargle periodogram
        phase = ((time-time[0])/pg.period_at_max_power.value) - np.floor((time-time[0])/pg.period_at_max_power.value)
        phase2 = [phase-1.0,phase]
        mag = [magnitude,magnitude]
        
        def plots(ax1,ax2,ax3):  #Defining a function that creates a plot with subplots: scatter plot of magnitude vs time, lomb-scargle periodogram, and a phase folded plot of the magnitude at the period at max power
            lc.scatter(ax=ax1)
            pg.plot(ax=ax2)
            ax3.scatter(phase2,mag,marker = ".",s = 5)

        if pg.period_at_max_power.value > max:  #Appending 'no' to the list if the period is greater than the maximum value
            cleaned.append('no')

        elif pg.period_at_max_power.value < min: #Appending 'no' to the list if the period is less than the minimum value
            cleaned.append('no')

        else:
        
            window = tk.Tk()  #Creating a GUI using tkinter
            window.title(f"Is this a {dataset['Type'][i]}? Index {i}")
            window.geometry("500x1000")

            fig, (ax1,ax2,ax3) = plt.subplots(3,1,figsize=(18,25))

            ax2.text(0.5, 0.8, f'Within (0.025,20) period = {pg.period_at_max_power}', transform=ax2.transAxes, fontsize=15)

            ax1.invert_yaxis()
            ax3.invert_yaxis()
            ax1.set_ylabel('Magnitude')
            ax3.set_xlabel('Phase')
            ax3.set_ylabel('Magnitude')

            plots(ax1,ax2,ax3)
            plt.suptitle(f"{dataset['Type'][i]}",fontsize = 30)

            canvas = FigureCanvasTkAgg(fig, master=window)
            canvas.draw()

            canvas_widget = canvas.get_tk_widget()
            canvas_widget.pack(side=tk.TOP, fill=tk.BOTH, expand=1)

            def yes_command():  #Defining a button that will append "yes" to the list if the button is pressed
                cleaned.append('yes')
                if not os.path.exists(f"{foldername}/{seccam}_{dataset['Type'][i]}_images"):
                    os.makedirs(f"{foldername}/{seccam}_{dataset['Type'][i]}_images")
                plt.savefig(f"{foldername}/{seccam}_{dataset['Type'][i]}_images/{TICID}_{SECTOR}_{CAMERA}_{CCD}.jpg")
                plt.close(fig)
                window.update_idletasks()
                window.destroy()
        
            def no_command(): #Defining a button that will append "no" to the list if the button is pressed
                cleaned.append('no')
                plt.close(fig)
                window.destroy()
        
            def on_key_press(event):  #The buttons can be finnicky so this block of code will allow the y key to be used in place of the 'yes' button and the n key in place of the 'no' button
                if event.keysym == 'y':
                    yes_button.invoke()
                elif event.keysym == 'n':
                    no_button.invoke()
    
            yes_button = ttk.Button(window, text="Yes", command=yes_command)
            yes_button.pack(side=tk.LEFT)
    
            no_button = ttk.Button(window, text="No", command=no_command)
            no_button.pack(side=tk.RIGHT)
    
            window.bind('<y>', on_key_press)
            window.bind('<n>', on_key_press)

            window.mainloop()
    
            os.remove(f"{TICID}_{SECTOR}_{CAMERA}_{CCD}.lc") #removing the file with the light curve data to take up less space
        
            if i == (len(dataset['TIC_ID'])-1):  #Creating a csv file with all the 'yes' and 'no' values
                with open(f'{newfilename}.csv', 'w', newline='') as csvfile:
                    writer = csv.writer(csvfile)
                    for item in cleaned:
                        writer.writerow([item])
    ;

In [ ]:
#This is a function that outputs the columns that cause the highest accuracy (or silhouette for kmeans) score when put through a machine learning classifier
#'dataset' is a pandas dataset that includes all 'types' any columns necessary for machine learning 
#cols is a list of all column names necessary for machine learning that are in the dataset
#classifier is a machine learning classifier, example: RandomForestClassifier(random_state=20,n_estimators=25)
#numcols is the maximum number of columns that you want to find the highest accuracy score with, recommended to choose 3 or less because otherwise it will take very long for the code to run, but 5 is the maximum
#silhouette is either True or False depending on whether or not a silhouette score is needed instead of an accuracy score
def bestcols(dataset,cols,classifier,numcols,silhouette):
    start_time = datetime.now()

    dict2 = {} #Empty dictionaries are initialized for each number of columns that can be chosen
    dict3 = {}
    dict4 = {}
    dict5 = {}

    for val in cols: #This first chunk of code loops through combinations of 2 columns, forming a training and testing set for each
        for val2 in cols:
            vals2 = [val,val2]
            if len(vals2) == len(set(vals2)) and numcols >= len(vals2):
                X = dataset[vals2]
                X = X.apply(pd.to_numeric)
                Y = dataset['Type']
                X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.25, train_size=0.75)
                
                cl = classifier
                with warnings.catch_warnings():
                    warnings.filterwarnings("ignore", category=ConvergenceWarning, module="sklearn")
                    cl.fit(X_train, y_train) #Each combinaiton of columns is run through the machine learning classifier
                if silhouette == False: #If silhouette is False, accuracy score is used
                    ypred = cl.predict(X_test)
                    score = metrics.accuracy_score(y_test, ypred)
                if silhouette == True: #If silhouette is True, silhouette score is used
                    ypred = cl.fit_predict(X_test)
                    score = silhouette_score(X_test, ypred)

                dict2.update({score:vals2}) #All the scores for each combination of 2 columns is added to dict2
                
            for val3 in cols: #Previous comments apply to next 3 chunks of code, this one is for combinations of 3 columns and so on
                vals3 = [val,val2,val3]
                if len(vals3) == len(set(vals3)) and numcols >= len(vals3):
                
                    X = dataset[vals3]
                    X = X.apply(pd.to_numeric)
                    Y = dataset['Type']
                    X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.25, train_size=0.75)
                
                    cl = classifier
                    with warnings.catch_warnings():
                        warnings.filterwarnings("ignore", category=ConvergenceWarning, module="sklearn")
                        cl.fit(X_train, y_train)
                    if silhouette == False:
                        ypred = cl.predict(X_test)
                        score = metrics.accuracy_score(y_test, ypred)
                    if silhouette == True:
                        ypred = cl.fit_predict(X_test)
                        score = silhouette_score(X_test, ypred)

                    dict3.update({score:vals3})
                
                for val4 in cols:
                    vals4 = [val,val2,val3,val4]
                    if len(vals4) == len(set(vals4)) and numcols >= len(vals4):
                    
                        X = dataset[vals4]
                        X = X.apply(pd.to_numeric)
                        Y = dataset['Type']
                        X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.25, train_size=0.75)
                
                        cl = classifier
                        with warnings.catch_warnings():
                            warnings.filterwarnings("ignore", category=ConvergenceWarning, module="sklearn")
                            cl.fit(X_train, y_train)
                        if silhouette == False:
                            ypred = cl.predict(X_test)
                            score = metrics.accuracy_score(y_test, ypred)
                        if silhouette == True:
                            ypred = cl.fit_predict(X_test)
                            score = silhouette_score(X_test, ypred)

                        dict4.update({score:vals4})
                        
                    for val5 in cols:
                        vals5 = [val,val2,val3,val4,val5]
                        if len(vals4) == len(set(vals5)) and numcols >= len(vals5):
                    
                            X = dataset[vals5]
                            X = X.apply(pd.to_numeric)
                            Y = dataset['Type']
                            X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.25, train_size=0.75)
                
                            cl = classifier
                            with warnings.catch_warnings():
                                warnings.filterwarnings("ignore", category=ConvergenceWarning, module="sklearn")
                                cl.fit(X_train, y_train)
                            if silhouette == False:
                                ypred = cl.predict(X_test)
                                score = metrics.accuracy_score(y_test, ypred)
                            if silhouette == True:
                                ypred = cl.fit_predict(X_test)
                                score = silhouette_score(X_test, ypred)

                            dict5.update({score:vals5})
                    
    if numcols >= len(vals2):  #These chunks of code find the combination of 2,3,4 and 5 columns in the dictionaries that have the highest accuracy/silhouette scores, depending on what numcols is, and prints the score along with the column names
        scores2 = list(dict2.keys())
        maxscore2 = max(scores2)
        cols2 = dict2[maxscore2]
        print(maxscore2,cols2)
    
    if numcols >= len(vals3):           
        scores3 = list(dict3.keys())
        maxscore3 = max(scores3)
        cols3 = dict3[maxscore3]
        print(maxscore3,cols3)
    
    if numcols >= len(vals4):
        scores4 = list(dict4.keys())
        maxscore4 = max(scores4)
        cols4 = dict4[maxscore4]
        print(maxscore4,cols4)
        
    if numcols >= len(vals5):
        scores5 = list(dict5.keys())
        maxscore5 = max(scores5)
        cols5 = dict5[maxscore5]
        print(maxscore5,cols5)

    X = dataset.drop(columns = ['Type']) #This runs the machine learning classifier with all columns and prints the score under the combinations of fever columns
    X = X.apply(pd.to_numeric)
    Y = dataset['Type']
    X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=.25, train_size=.75)

    cl = classifier
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=ConvergenceWarning, module="sklearn")
        cl.fit(X_train, y_train)
    ypred = cl.predict(X_test)
    score = metrics.accuracy_score(y_test, ypred)

    print(score,"All columns")

    end_time = datetime.now() #The time taken to run the cell is printed for reference, because with high numcols it can take a long time
    print('Duration: {}'.format(end_time - start_time))

In [ ]:
#This function finds 'Vmag' - 'Kmag' and 'w1mag' - 'w4mag' and saves them as new columns in the dataset
#'dataset' is a pandas dataframe that includes the columns 'Vmag','Kmag','w1mag', and 'w4mag'
def magdiff(dataset):
    VmKmag = [] 
    for i in range(len(dataset['Kmag'])): #For each index in the dataset, the value for Kmag is subtracted from the value for Vmag and appended to a list
        VmKmag.append(float(dataset['Vmag'][i]) - float(dataset['Kmag'][i]))
    
    w1mw4mag = []
    for i in range(len(dataset['w1mag'])): #For each index in the dataset, the value for w4mag is subtracted from the value for w1mag and appended to a list
        w1mw4mag.append(float(dataset['w1mag'][i]) - float(dataset['w4mag'][i]))

    dict = {'VmKmag': VmKmag, 'w1mw4mag': w1mw4mag} #The lists of values are given their corresponding column names in a dictionary, then converted to a pandas dataframe
   
    calcs = pd.DataFrame(dict)
   
    datasetwcalcs = pd.concat([dataset, calcs], axis=1) #The new columns 'VmKmag' and 'w1mw4mag' are added to the original dataset, then the columns 'w1mag','w4mag','Vmag','Kmag' are dropped
    datasetwcalcs = datasetwcalcs.drop(['w1mag','w4mag','Vmag','Kmag'], axis=1)
    return datasetwcalcs #The new dataset with the necessary magnitude differences is returned

In [ ]:
#This function cleans a dataset that was put through the "clean" function by adding the CSV file returned by the "clean" function as a column to the dataset used, then dropping all the indices where the new column has "no"
#'dataset' is a pandas dataframe that was put through the "clean" function, or the entire dataset with all types as long as .head() was used for the single type dataset put into the "clean" function
#'type' is the star type of the dataset put through the "clean" function
#'typecleanedfilename' is the name of the CSV file that was returned by the 'clean' function, containing the strings 'yes' and 'no'
def cleaneddata(dataset,type,typecleanedfilename):
    
    nottypes = [] #If the entire original dataset is used as 'dataset', the indices where the 'Type' column is not equal to the type inputted will be appended to this list

    if type == 'MISC':
        for i in range(len(dataset['Type'])):
            if 'MISC' not in dataset['Type'][i]:
                nottypes.append(i)

    if type == 'CST':
        for i in range(len(dataset['Type'])):
            if 'CST' not in dataset['Type'][i]:
                nottypes.append(i)

    if type == 'RR':
        for i in range(len(dataset['Type'])):
            if 'RR' not in dataset['Type'][i]:
                nottypes.append(i)

    if type == 'EC':
        for i in range(len(dataset['Type'])):
            if 'EC' not in dataset['Type'][i]:
                if 'EW' not in dataset['Type'][i]:
                    nottypes.append(i)

    if type == 'ED':   
        for i in range(len(dataset['Type'])):             
            if 'ED' not in dataset['Type'][i]:
                if 'EA' not in dataset['Type'][i]:
                    if 'ESD' not in dataset['Type'][i]:
                        if 'EB' not in dataset['Type'][i]:
                            nottypes.append(i)

    if type == 'DSCT': 
        for i in range(len(dataset['Type'])):
            if 'DSCT' not in dataset['Type'][i]:
                if 'HADS' not in dataset['Type'][i]:
                    nottypes.append(i)

    if type == 'DCEP': 
        for i in range(len(dataset['Type'])):
            if 'DCEP' not in dataset['Type'][i]:
                nottypes.append(i)

    if type == 'GDOR': 
        for i in range(len(dataset['Type'])):     
            if 'GDOR' not in dataset['Type'][i]:
                nottypes.append(i)

    types = dataset.drop(nottypes) #This line of code drops all indices where the "Type" column is not equal to the type inputted

    for val in types.Type.unique():
        types.replace(to_replace=val, value=type, inplace=True)

    testing = pd.read_csv(f"{typecleanedfilename}.csv", sep=',',header = None) #The CSV of 'yes' and 'no' strings is loaded in as a pandas column

    types = types.head(len(testing)) #If the entire dataset was used, this ensures that it is the same length as the CSV file
    types = types.reset_index()
    types = types.drop('index',axis = 1)

    cols = pd.concat([testing, types], axis=1) #This last chunk of code concatenates the dataset and the "yes"/"no" column, and then drops the incides where the column from the CSV files says "no"
    nos = []
    for i in range(len(cols[0])):
        if cols[0][i] == 'no':
            nos.append(i)
        
    allyes = cols.drop(nos)
    allyes = allyes.reset_index()
    cleandata = allyes.drop('index',axis = 1)
    
    return cleandata #The cleaned dataset of the single star type is returned (Indices where the CSV file returned from the 'clean' function was "yes")


In [ ]:
#This is just a quick function that puts a dataset through a classifier, then concatenates the column of predicted types to the dataset
#'dataset' is a pandas dataframe with any columns needed for machine learning
#classifier is a variable name for a machine learning classifier that was previously run with a training dataset
#cols is a list of all column names necessary for machine learning that are in the dataset
def resulttypes(dataset,classifier,cols):
    predtypes = classifier.predict(dataset[cols])
    predtypes = pd.DataFrame({'PredType': predtypes})
    
    results = pd.concat([predtypes,dataset], axis=1)
    return results #Returns the inputted pandas dataframe with an added column of the predicted types from the machine learning classifier

In [ ]:
#This function takes a dataset that was previously returned by the 'resulttypes' function, and creates jpeg plots for each predicted type, similar to the 'clean' function
#'dataset' is the pandas dataframe returned by the 'resulttypes' function that includes columns 'PredType','TIC_ID','Sector','Camera',and 'CCD'
#'seccam' is a name for the sector and camera that the dataset is from, example 'sec15cam3'
#'foldername' is the name of the main folder that you would like all the folders for each star type containing the light curves to go into.
#Right now, only sectors 1,2,3,4,5,15,16, and 17 will work with this function
def resultplots(dataset,seccam,foldername):

    for i in range(len(dataset['TIC_ID'])): #Looping to get a light curve for each index in the dataset
    
        TICID = dataset['TIC_ID'][i]
        SECTOR = dataset['Sector'][i]
        CAMERA = dataset['Camera'][i]
        CCD = dataset['CCD'][i]
        if SECTOR in ['sector01','sector02','sector03','sector04','sector05']: #A different url is used to download the lightcurve data for sectors 1-5 vs sectors 15-17
            url = f"http://astro.phy.vanderbilt.edu/~oelkerrj/tess_ffi/{SECTOR}/clean/{TICID}_{SECTOR}_{CAMERA}_{CCD}.lc"
        if SECTOR in ['sector15','sector16','sector17']:
            url = f"http://astro.phy.vanderbilt.edu/~oelkerrj/tess_ffi/{SECTOR}/detrend/{TICID}_{SECTOR}_{CAMERA}_{CCD}.lc"
        
        os.system(f"wget {url} -O {TICID}_{SECTOR}_{CAMERA}_{CCD}.lc > /dev/null 2>&1")
    
        try:
            data = pd.read_csv(f"{TICID}_{SECTOR}_{CAMERA}_{CCD}.lc", delimiter=' ', header=None, names=['time', 'magnitude', 'error'])
        except pd.errors.ParserError:
            os.remove(f"{TICID}_{SECTOR}_{CAMERA}_{CCD}.lc") #Removing light curve files that cause errors so that the cell continues to run instead of being stopped
            continue
        data = data.to_numpy(dtype=float) #Ensuring that the datatype is float

        if np.any(np.isnan(data)) or np.any(np.isinf(data)): #Checking if there are any nan or inf in the light curve data and skipping the index if there are
            os.remove(f"{TICID}_{SECTOR}_{CAMERA}_{CCD}.lc")
            continue
            
        time = data[:,0]
        magnitude = data[:,1]

        print(time)
        print(magnitude)
    
        lc = lk.LightCurve(time,magnitude).remove_outliers(sigma=5.0,return_mask=False)
        
        pg = lc.to_periodogram(minimum_period=0.025*u.day, maximum_period=20*u.day,oversample_factor=10) 
        phase = ((time-time[0])/pg.period_at_max_power.value) - np.floor((time-time[0])/pg.period_at_max_power.value) #Using lightkurve to find the period with max power using a lomb-scargle periodogram
        phase2 = np.concatenate((phase - 1.0, phase))
        mag = np.concatenate((magnitude, magnitude))
        
        def plots(ax1,ax2,ax3): #Defining a function that creates a plot with subplots: scatter plot of magnitude vs time, lomb-scargle periodogram, and a phase folded plot of the magnitude at the period at max power
            lc.scatter(ax=ax1)
            pg.plot(ax=ax2)
            ax3.scatter(phase2,mag,marker = ".",s = 5)

        fig, (ax1,ax2,ax3) = plt.subplots(3,1,figsize=(18,25))

        ax2.text(0.5, 0.8, f'Within (0.025,20) period = {pg.period_at_max_power}', transform=ax2.transAxes, fontsize=15)

        ax1.invert_yaxis()
        ax3.invert_yaxis()

        plots(ax1,ax2,ax3)
        plt.suptitle(f"{dataset['PredType'][i]}",fontsize = 30)

        if not os.path.exists(f"{foldername}/{seccam}_pred_{dataset['PredType'][i]}"):
            os.makedirs(f"{foldername}/{seccam}_pred_{dataset['PredType'][i]}") #Creating a folder inside the main folder for each specific star type if the folders do not already exist

        plt.suptitle(f"{dataset['PredType'][i]}",fontsize = 20)
        plt.savefig(f"{foldername}/{seccam}_pred_{dataset['PredType'][i]}/{TICID}_{SECTOR}_{CAMERA}_{CCD}.jpg") #Saving the jpeg image of the light curve plots
        plt.close(fig)
    
        os.remove(f"{TICID}_{SECTOR}_{CAMERA}_{CCD}.lc") #removing the file with the light curve data to take up less space
    ;

In [ ]:
#This is an example of how I read in the data for a sector
#The typenames file is downloaded from VisieR (choose 'list of targets' then check the boxes for 'single table' and 'add your input as first column'. Then on the left choose 'max: '1' and ;-Separated-Values instead of HTML Table, then make sure only 'Type' is checked under simple constraint) after inputting a space separated file with 'ra' and 'dec' from filtergraph
#The csv file for each camera is from filtergraph with the columns 'TIC_ID','Sector','Camera','CCD','Vmag','Kmag','w1mag','w4mag','Teff','Jstet','RMS_60m','LS_Period','LS_SNR','BLS_Period','BLS_SDE' checked and downloaded as a comma-separated file

types1 = pd.read_csv('typenames_sector1.tsv', sep=';', comment='#',skip_blank_lines = False,header = 1)
types1 = types1.drop([0,1])
types1 = types1.reset_index()
types1 = types1.drop('index',axis = 1)

l = []
for i in range(len(types1['_1'])):
    if type(types1['_1'][i]) != str:
        l.append(i)
        
types1 = types1.drop(l)
types1 = types1.reset_index()
cropped1 = types1.drop('index',axis = 1)

cam1 = pd.read_csv('Sector1_camera1.csv', sep=',', comment='#', dtype = str)
cam2 = pd.read_csv('Sector1_camera2.csv', sep=',', comment='#', dtype = str)
cam3 = pd.read_csv('Sector1_camera3.csv', sep=',', comment='#', dtype = str)
cam4 = pd.read_csv('Sector1_camera4.csv', sep=',', comment='#', dtype = str)
cams = pd.concat([cam1,cam2,cam3,cam4])
cams = cams.reset_index()
alldata = cams.drop('index',axis = 1)
alldata = alldata.drop('Unnamed: 17',axis = 1)

Sector1 = pd.concat([cropped1, alldata], axis=1)

m = []
for i in range(len(Sector1['Type'])):
    if type(Sector1['Type'][i]) != str:
        m.append(i)
        
Sector1 = Sector1.drop(m)
Sector1 = Sector1.dropna(axis=0)
Sector1 = Sector1.reset_index()
Sector1 = Sector1.drop('index',axis = 1)

Sector1

In [ ]:
#This shows how the 'septypes' function separates all the star types into different pandas dataframes
MISC1,CST1,RR1,EC1,ED1,DSCT1,DCEP1,GDOR1 = septypes(Sector1)

In [ ]:
#Example of how to use the "clean" function without constraining the period (just use very small and very large numbers)
%%capture
clean(GDOR,'GDOR_all','GDOR_images',100,0.001)
;

In [ ]:
#Using the 'magdiff' function on each of the type dataframes (This could have been done on the entire sector)
diffCST1 = magdiff(CST1)
diffMISC1 = magdiff(MISC1)
diffRR1 = magdiff(RR1)
diffEC1 = magdiff(EC1)
diffED1 = magdiff(ED1)
diffDSCT1 = magdiff(DSCT1)
diffDCEP1 = magdiff(DCEP1)
diffGDOR1 = magdiff(GDOR1)

diffMISC1

In [ ]:
#Example of cleaning each of the datasets with their corresponding CSV files from the "clean" function
#GDOR and CST were more complicated because I used the "clean" function on MISC a couple times to find more constant and gamma doradus stars that were within the MISC category

sec1cleanMISC = cleaneddata(diffMISC1,'MISC','MISCto999')
sec1cleanRR = cleaneddata(diffRR1,'RR','RRto999')
sec1cleanEC = cleaneddata(diffEC1,'EC','ECto999')
sec1cleanED = cleaneddata(diffED1,'ED','EDto999')
sec1cleanDSCT = cleaneddata(diffDSCT1,'DSCT','DSCT_all')
sec1cleanDCEP = cleaneddata(diffDCEP1,'DCEP','DCEPto999')

sec1cleanCST1 = cleaneddata(diffCST1,'CST','CST_all')
sec1cleanGDOR1 = cleaneddata(diffGDOR1,'GDOR','GDOR_all')

sec1cleanCST2 = cleaneddata(diffMISC1,'MISC','const_new')
sec1cleanGDOR2 = cleaneddata(diffMISC1,'MISC','GDOR_new')

for val in sec1cleanCST2.Type.unique():
        sec1cleanCST2.replace(to_replace=val, value='CST', inplace=True)

for val in sec1cleanGDOR2.Type.unique():
        sec1cleanGDOR2.replace(to_replace=val, value='GDOR', inplace=True)

sec1cleanGDOR = pd.concat([sec1cleanGDOR1, sec1cleanGDOR2])
sec1cleanGDOR = sec1cleanGDOR.reset_index()
sec1cleanGDOR = sec1cleanGDOR.drop('index',axis = 1)

sec1cleanCST = pd.concat([sec1cleanCST1, sec1cleanCST2])
sec1cleanCST = sec1cleanCST.reset_index()
sec1cleanCST = sec1cleanCST.drop('index',axis = 1)

In [ ]:
#Creating a mixed dataset of all the types concatenated to use for machine learning
clean_sec1 = pd.concat([sec1cleanGDOR,sec1cleanDCEP,sec1cleanDSCT,sec1cleanED,sec1cleanRR,sec1cleanEC,sec1cleanMISC,sec1cleanCST])
clean_sec1 = clean_sec1.reset_index()
clean_sec1 = clean_sec1.drop('index',axis = 1)
clean_sec1

clean_sec1_mix = clean_sec1.sample(frac = 1)

sec1_ml = clean_sec1_mix[['Type','Teff','Jstet','RMS_60m','LS_Period','LS_SNR','BLS_Period','BLS_SDE','VmKmag','w1mw4mag']]
sec1_ml

In [ ]:
#Example of how the 'bestcols' function works
cols = ['Teff','Jstet','RMS_60m','LS_Period','LS_SNR','BLS_Period','BLS_SDE','VmKmag','w1mw4mag']

bestcols(sec1_ml,cols,RandomForestClassifier(random_state=20,n_estimators=25),3,False)

In [ ]:
#Creating the variable for a random forest classifier to use for 'resulttypes' function later on
X = sec1_ml.drop('Type',axis = 1)
X = X.apply(pd.to_numeric)
Y = sec1_ml['Type']
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.3, train_size=0.7)

forest = RandomForestClassifier(n_estimators=30)
forest.fit(X_train, y_train)

ypredfor = forest.predict(X_test)

score = metrics.accuracy_score(y_test, ypredfor)

score


In [ ]:
#Checking the classifier's confusion matrix
metrics.confusion_matrix(y_test, ypredfor)

In [ ]:
#Bringing in sector 15 cameras 1 through 3 in a similar way to sector 1 earlier
sec15cam1to3types = pd.read_csv('Sector15_cam1-3_typenames_07-30-24.tsv', sep=';', comment='#',skip_blank_lines = False,header = 1)
sec15cam1to3types = sec15cam1to3types.drop([0,1])
sec15cam1to3types = sec15cam1to3types.reset_index()
sec15cam1to3types = sec15cam1to3types.drop('index',axis = 1)

l = []
for i in range(len(sec15cam1to3types['_1'])):
    if type(sec15cam1to3types['_1'][i]) != str:
        l.append(i)
        
sec15cam1to3types = sec15cam1to3types.drop(l)
sec15cam1to3types = sec15cam1to3types.reset_index()
sec15cam1to3types = sec15cam1to3types.drop('index',axis = 1)


sec15cam3data = pd.read_csv('Sector15_camera3_07-24-24.csv', sep=',', comment='#', dtype = str)
sec15cam1to2data = pd.read_csv('Sector15_cam1-2_07-30-24.csv', sep=',', comment='#', dtype = str)
sec15cam1to3data = pd.concat([sec15cam1to2data,sec15cam3data])
sec15cam1to3data = sec15cam1to3data.reset_index()
sec15cam1to3data = sec15cam1to3data.drop('index',axis = 1)

sec15cam1to3data = sec15cam1to3data.drop('Unnamed: 15',axis = 1)

sec15cam1to3 = pd.concat([sec15cam1to3types, sec15cam1to3data], axis=1)

m = []
for i in range(len(sec15cam1to3['Type'])):
    if type(sec15cam1to3['Type'][i]) != str:
        m.append(i)
        
sec15cam1to3 = sec15cam1to3.drop(m)
sec15cam1to3 = sec15cam1to3.dropna(axis=0)
sec15cam1to3 = sec15cam1to3.drop('_1',axis = 1)
sec15cam1to3 = sec15cam1to3.reset_index()
sec15cam1to3 = sec15cam1to3.drop('index',axis = 1)

sec15cam1to3

In [ ]:
#Renaming the columns to match the ones for sector 1
sec15cam1to3.rename(columns={'vmag': 'Vmag','kmag': 'Kmag','teff':'Teff','Jstet_2':'Jstet','RMSBin_60.0_3':'RMS_60m','LS_Period_1_4':'LS_Period','LS_SNR_1_4':'LS_SNR','BLS_Period_1_5':'BLS_Period','BLS_SDE_1_5':'BLS_SDE'}, inplace=True)
sec15cam1to3

In [ ]:
#Using magdiff on the entire dataset
sec15cam1to3diff = magdiff(sec15cam1to3)
sec15cam1to3diff

In [ ]:
#Using the 'resulttypes' function with the previously created 'forest' classifier to predict the types of the stars in
mlcols = ['Teff','Jstet','RMS_60m','LS_Period','LS_SNR','BLS_Period','BLS_SDE','VmKmag','w1mw4mag']
sec15cam3pred = resulttypes(sec15cam3diff,forest,mlcols)
sec15cam3pred

In [ ]:
#Using 'resultplots' function to plot out the predicted types with their lightcurves and save them as jpeg files for comparison
%%capture
resultplots(sec15cam3pred,'sec15cam3','Sector15_Camera3_Predicted')
;

In [ ]:
#Continuing on with more testing to see how accurate the 'forest' classifier is on sector 15
#Using septypes similarly to sector 1
MISC15,CST15,RR15,EC15,ED15,DSCT15,DCEP15,GDOR15 = septypes(sec15cam1to3diff)

In [ ]:
#Looking at the length of the files to see if they are different enough to affect the machine learning (77402 is way higher than 151)
print(len(MISC15))
print(len(CST15))
print(len(RR15))
print(len(EC15))
print(len(ED15))
print(len(DSCT15))
print(len(DCEP15))
print(len(GDOR15))

In [ ]:
#Creating a machine learning dataset with all indices
sec15types = pd.concat([MISC15,CST15,RR15,EC15,ED15,DSCT15,DCEP15,GDOR15])
sec15types_mix = sec15types.sample(frac = 1)
sec15types_mix = sec15types_mix.reset_index()
sec15types_mix = sec15types_mix.drop('TIC_ID',axis = 1)
sec15types_mix = sec15types_mix.drop('Sector',axis = 1)
sec15types_mix = sec15types_mix.drop('Camera',axis = 1)
sec15types_mix = sec15types_mix.drop('CCD',axis = 1)
sec15types_mix = sec15types_mix.drop('index',axis = 1)
sec15types_mix


In [ ]:
#Using 150 random indices from each type to create an even dataset
sec15typescut = pd.concat([MISC15.sample(150),CST15.sample(150),RR15.sample(150),EC15.sample(150),ED15.sample(150),DSCT15.sample(150),DCEP15.sample(150),GDOR15.sample(150)])
sec15typescut = sec15typescut.sample(frac = 1)
sec15typescut = sec15typescut.reset_index()
sec15typescut = sec15typescut.drop('TIC_ID',axis = 1)
sec15typescut = sec15typescut.drop('Sector',axis = 1)
sec15typescut = sec15typescut.drop('Camera',axis = 1)
sec15typescut = sec15typescut.drop('CCD',axis = 1)
sec15typescut = sec15typescut.drop('index',axis = 1)
sec15typescut

In [ ]:
#Finding the best columns to use for the dataset with 150 of each type, it ends up extremely low so I think the dataset definitely needs to be cleaned
cols = ['Teff','Jstet','RMS_60m','LS_Period','LS_SNR','BLS_Period','BLS_SDE','VmKmag','w1mw4mag']

bestcols(sec15typescut,cols,RandomForestClassifier(random_state=20,n_estimators=25),3,False)

In [ ]:
#Finding the best columns to use for the dataset with all values of each type, I think the huge about of binaries classified is skewing the accuracy to look better than it actually is
bestcols(sec15types_mix,cols,RandomForestClassifier(random_state=20,n_estimators=25),3,False)

In [ ]:
#Cleaning sector 15 similarly to sector 1 using the 'clean' function for each star type
%%capture
clean(MISC15.head(176),'MISC15_all','Sector15_images','sec15cam1to3',100,0)

In [ ]:
%%capture
clean(CST15,'CST15_all','Sector15_images','sec15cam1to3',100,0)

In [ ]:
#Running into an "Illegal type  for table item access" error here that I still need to fix
%%capture
clean(DCEP15,'DCEP15_all','Sector15_images','sec15cam1to3',100,1.4)

In [ ]:
%%capture
clean(GDOR15,'GDOR15_all','Sector15_images','sec15cam1to3',100,0)

In [ ]:
%%capture
clean(RR15.head(100),'RR15to999','Sector15_images','sec15cam1to3',100,0)

In [ ]:
%%capture
clean(GDOR15,'GDOR15_all','Sector15_images','sec15cam1to3',100,0)